# Analysis

In [1]:
library(xts)
library(DBI)
con <- dbConnect(odbc::odbc(), "JupyterLab", timeout = 10)

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric




## RPNA story count

In this section, we build 4 time series of story count per day:
- total_count
- macro_count - stories containing no mention of any company
- comp_count - stories containing only mentions of companies
- mix_count - stories containg both mentions

In [2]:
qry <-
"SELECT COUNT(DISTINCT RP_STORY_ID) AS NO_STORIES
 FROM dbo.RPNA_WSJ;"

dbGetQuery(con, qry)

NO_STORIES
<int>
845222


The Wall Street Journal published 845'222 stories from January 1, 2001 to August 31, 2019.

In [3]:
qry <-
"SELECT
    ENTITY_TYPE,
    COUNT(DISTINCT RP_STORY_ID) AS NO_STORIES,
    100.0 * COUNT(DISTINCT RP_STORY_ID) / (SELECT COUNT(DISTINCT RP_STORY_ID) FROM dbo.RPNA_WSJ) AS FRAC_STORIES
FROM dbo.RPNA_WSJ
GROUP BY ENTITY_TYPE
ORDER BY NO_STORIES DESC;"

dbGetQuery(con, qry)

ENTITY_TYPE,NO_STORIES,FRAC_STORIES
<chr>,<int>,<dbl>
COMP,678952,80.3282451
ORGA,416262,49.2488364
CMDT,125606,14.8607112
PLCE,10517,1.2442885
CURR,1286,0.1521494


We see that 80% of the stories are related to at least one company. Also, organizations (50%) and commodities (15%) make up most of the "macro" content. Places (1.24%) and currencies (0.15%) are rarely covered.

In [4]:
qry <-
"WITH
 not_comp AS
 (
    SELECT RP_STORY_ID
    FROM dbo.RPNA_WSJ
    WHERE ENTITY_TYPE<>'COMP'
 ),
 only_comp AS
 (
    SELECT RP_STORY_ID
    FROM dbo.RPNA_WSJ
    WHERE ENTITY_TYPE='COMP'
 )

 SELECT
    t.[TYPE],
    t.NO_STORIES,
    100.0 * t.NO_STORIES / SUM(NO_STORIES) OVER() AS FRAC_STORIES
 FROM (
    SELECT
        'Only companies mentioned' AS [TYPE],
        COUNT(DISTINCT RP_STORY_ID) AS [NO_STORIES]
    FROM dbo.RPNA_WSJ
    WHERE RP_STORY_ID NOT IN (SELECT * FROM not_comp)

    UNION ALL

    SELECT
        'No companies mentioned',
        COUNT(DISTINCT RP_STORY_ID)
    FROM dbo.RPNA_WSJ
    WHERE RP_STORY_ID NOT IN (SELECT * FROM only_comp)

    UNION ALL

    SELECT
        'Both mentioned',
        COUNT(DISTINCT RP_STORY_ID)
    FROM dbo.RPNA_WSJ
    WHERE RP_STORY_ID IN (SELECT * FROM not_comp)
      AND RP_STORY_ID IN (SELECT * FROM only_comp)
 ) t
 ORDER BY FRAC_STORIES DESC;"

dbGetQuery(con, qry)

TYPE,NO_STORIES,FRAC_STORIES
<chr>,<int>,<dbl>
Only companies mentioned,371901,44.00039
Both mentioned,307051,36.32785
No companies mentioned,166270,19.67175


Going forward, we will define stories containing no mention of a company as "macro" news. All stories mentioning at least one company will be termed "equity" news. Hence, a more precise distinction would be between "equity" and "non-equity" news.

**TBD:** Decide whether to define macro news as no equity or no equity + both.

In [5]:
# Get count of total stories per day
qry <-
"DECLARE @time_shift int = -6;

 SELECT
    CAST(DATEADD(HOUR, @time_shift, TIMESTAMP_EST) AS date) AS [DATE],
    COUNT(DISTINCT RP_STORY_ID) AS total_count
 FROM dbo.RPNA_WSJ
 GROUP BY CAST(DATEADD(HOUR, @time_shift, TIMESTAMP_EST) AS date)
 ORDER BY [date];"
df <- dbGetQuery(con, qry)
total_count <- xts(df[,2], order.by=df$DATE) # No need to set time zone as everything is done in SQL and we only work with dates in R.
names(total_count) <- names(df)[2]
head(total_count)

           total_count
2001-01-01          95
2001-01-03         104
2001-01-04         136
2001-01-07         145
2001-01-08         124
2001-01-09         121

In [6]:
# Get count of non-company stories per day
qry <-
"DECLARE @time_shift int = -6;

 SELECT
    CAST(DATEADD(HOUR, @time_shift, TIMESTAMP_EST) AS date) AS [DATE],
    COUNT(DISTINCT RP_STORY_ID) AS macro_count
 FROM dbo.RPNA_WSJ
 WHERE RP_STORY_ID NOT IN (
    SELECT RP_STORY_ID
    FROM dbo.RPNA_WSJ
    WHERE ENTITY_TYPE='COMP'
 )
 GROUP BY CAST(DATEADD(HOUR, @time_shift, TIMESTAMP_EST) AS date)
 ORDER BY [date];"
df <- dbGetQuery(con, qry)
macro_count <- xts(df[,2], order.by=df$DATE) # No need to set time zone as everything is done in SQL and we only work with dates in R.
names(macro_count) <- names(df)[2]
head(macro_count)

           macro_count
2001-01-01           9
2001-01-03          13
2001-01-04          17
2001-01-07          20
2001-01-08          15
2001-01-09          13

In [7]:
# Get count of companies-only stories per day
qry <-
"DECLARE @time_shift int = -6;

 SELECT
    CAST(DATEADD(HOUR, @time_shift, TIMESTAMP_EST) AS date) AS [DATE],
    COUNT(DISTINCT RP_STORY_ID) AS comp_count
 FROM dbo.RPNA_WSJ
 WHERE RP_STORY_ID NOT IN (
    SELECT RP_STORY_ID
    FROM dbo.RPNA_WSJ
    WHERE ENTITY_TYPE<>'COMP'
 )
 GROUP BY CAST(DATEADD(HOUR, @time_shift, TIMESTAMP_EST) AS date)
 ORDER BY [date];"
df <- dbGetQuery(con, qry)
comp_count <- xts(df[,2], order.by=df$DATE) # No need to set time zone as everything is done in SQL and we only work with dates in R.
names(comp_count) <- names(df)[2]
head(comp_count)

           comp_count
2001-01-01         49
2001-01-03         44
2001-01-04         63
2001-01-07         62
2001-01-08         60
2001-01-09         68

In [8]:
# Merge counts
news <- merge(total_count, macro_count, fill=0, join="outer")
news <- merge(news, comp_count, fill=0, join="outer")

# Mix_count contains stories that include both companies and non-company entities
news$mix_count <- news$total_count - (news$macro_count + news$comp_count)
head(news)

# Housekeeping
rm(total_count, macro_count, comp_count, df)

           total_count macro_count comp_count mix_count
2001-01-01          95           9         49        37
2001-01-03         104          13         44        47
2001-01-04         136          17         63        56
2001-01-07         145          20         62        63
2001-01-08         124          15         60        49
2001-01-09         121          13         68        40

In [9]:
str(news)
news[1:20,]
summary(news)

An 'xts' object on 2001-01-01/2019-08-30 containing:
  Data: int [1:6373, 1:4] 95 104 136 145 124 121 125 144 1 127 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:4] "total_count" "macro_count" "comp_count" "mix_count"
  Indexed by objects of class: [Date] TZ: UTC
  xts Attributes:  
 NULL


           total_count macro_count comp_count mix_count
2001-01-01          95           9         49        37
2001-01-03         104          13         44        47
2001-01-04         136          17         63        56
2001-01-07         145          20         62        63
2001-01-08         124          15         60        49
2001-01-09         121          13         68        40
2001-01-10         125          15         68        42
2001-01-11         144          19         70        55
2001-01-12           1           0          1         0
2001-01-14         127          24         50        53
2001-01-15          95           7         42        46
2001-01-16         136          15         68        53
2001-01-17         125          20         67        38
2001-01-18         154          27         66        61
2001-01-21         141          12         61        68
2001-01-22         116          14         61        41
2001-01-23         111          20         55   

     Index             total_count      macro_count       comp_count    
 Min.   :2001-01-01   Min.   :   1.0   Min.   :  0.00   Min.   :  0.00  
 1st Qu.:2005-12-11   1st Qu.:  74.0   1st Qu.: 13.00   1st Qu.: 26.00  
 Median :2010-06-26   Median : 113.0   Median : 21.00   Median : 46.00  
 Mean   :2010-06-02   Mean   : 132.6   Mean   : 26.09   Mean   : 58.36  
 3rd Qu.:2014-11-12   3rd Qu.: 158.0   3rd Qu.: 32.00   3rd Qu.: 71.00  
 Max.   :2019-08-30   Max.   :1120.0   Max.   :315.00   Max.   :674.00  
   mix_count     
 Min.   :  0.00  
 1st Qu.: 28.00  
 Median : 45.00  
 Mean   : 48.18  
 3rd Qu.: 61.00  
 Max.   :450.00  

## S&P 500 prices and returns

In this section, we will build time series of returns of companies that were included in the S&P 500 at any point between January 1, 2001 and August 31, 2019.